In [3]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import requests

In [4]:
spark = SparkSession.builder\
    .appName("DataRequestApp")\
    .master("spark://spark-master:7077")\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/08 00:01:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
import requests
response = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=256da2d742d5a5979790e6833447e4b4').json()

# API Reference

[TMDB](https://developer.themoviedb.org/reference/intro/getting-started)

In [6]:
df = spark.createDataFrame(response['results'])

In [7]:
df.show()

+-----+--------------------+--------------------+-------+-----------------+--------------------+--------------------+----------+--------------------+------------+--------------------+-----+------------+----------+
|adult|       backdrop_path|           genre_ids|     id|original_language|      original_title|            overview|popularity|         poster_path|release_date|               title|video|vote_average|vote_count|
+-----+--------------------+--------------------+-------+-----------------+--------------------+--------------------+----------+--------------------+------------+--------------------+-----+------------+----------+
|false|/zOpe0eHsq0A2NvNy...|[28, 878, 35, 10751]| 939243|               en|Sonic the Hedgehog 3|Sonic, Knuckles, ...|  3084.368|/d8Ryb8AunYAuycVK...|  2024-12-19|Sonic the Hedgehog 3|false|       7.787|      1566|
|false|/v9Du2HC3hlknAvGl...|        [28, 12, 53]| 539972|               en|   Kraven the Hunter|Kraven Kravinoff'...|   1785.56|/nrlfJoxP1EkBVE9

In [8]:
df_movies = df.select('title', 'release_date', 'overview', 'vote_average')

In [9]:
df_movies.show()

+--------------------+------------+--------------------+------------+
|               title|release_date|            overview|vote_average|
+--------------------+------------+--------------------+------------+
|Sonic the Hedgehog 3|  2024-12-19|Sonic, Knuckles, ...|       7.787|
|   Kraven the Hunter|  2024-12-11|Kraven Kravinoff'...|       6.605|
|             Moana 2|  2024-11-21|After receiving a...|       7.211|
|      Back in Action|  2025-01-15|Fifteen years aft...|       6.641|
|              Alarum|  2025-01-16|Two married spies...|         5.9|
|Sniper: The Last ...|  2025-01-21|To stop an arms d...|         6.4|
|Raiders of the Lo...|  1981-06-12|When Dr. Indiana ...|       7.927|
|            Babygirl|  2024-12-25|A high-powered CE...|       5.751|
|Venom: The Last D...|  2024-10-22|Eddie and Venom a...|       6.778|
|        Gladiator II|  2024-11-05|Years after witne...|         6.8|
|Mufasa: The Lion ...|  2024-12-18|Mufasa, a cub los...|       7.423|
|           Nosferat

In [10]:
df_movies.write.mode("overwrite").parquet("/home/jupyter/app/data/movies.parquet")

In [11]:
spark.stop()